<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Effects Model with Genomic Data</font> 
</div>

In [ ]:
using DataFrames,CSV,JWAS
using JWAS: Datasets,misc

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [2]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [3]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [4]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [6]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multiple Traits Analyses with Marker Information</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [8]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [9]:
model1 = build_model(model_equations,R);

In [10]:
set_covariate(model1,"age");

In [11]:
add_genotypes(model1,genofile,G,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [12]:
Pi=Dict([1.0; 1.0]=>0.7,[1.0;0.0]=>0.1,[0.0,1.0]=>0.1,[0.0; 0.0]=>0.1)
out = runMCMC(model1,data,Pi=Pi,chain_length=5000,methods="BayesC",
estimatePi=true,output_samples_frequency=5);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[10.9589 0.626223; 0.626223 1.09589].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          5
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.1,[1.0, 0.0]=>0.1,[1.0, 1.0]=>0.7,[0.0, 0.0]=>0.1)
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.0

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:04


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [13]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [14]:
file1="MCMC_samples_for_marker_effects_BW.txt"
file2="MCMC_samples_for_marker_effects_CW.txt";

In [15]:
get_breeding_values(model1,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV      │ PEV     │
├─────┼────┼──────────┼─────────┤
│ 1   │ S1 │ -1.92518 │ 10.5045 │
│ 2   │ D1 │ 3.19043  │ 34.1901 │
│ 3   │ O1 │ 4.66389  │ 67.3543 │
│ 4   │ O3 │ -5.92914 │ 43.9114 │            
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV       │ PEV      │
├─────┼────┼───────────┼──────────┤
│ 1   │ S1 │ -0.431824 │ 0.852433 │
│ 2   │ D1 │ 0.580859  │ 2.65969  │
│ 3   │ O1 │ 1.02509   │ 6.60053  │
│ 4   │ O3 │ -1.17413  │ 3.17826  │

In [16]:
samples4G=get_additive_genetic_variances(model1,file1,file2);

samples4R=out["MCMC samples for residual covariance matrix"];

samples4h2=get_heritability(reformat(samples4G),reformat(samples4R));

In [17]:
writedlm("out3.txt",samples4G)

In [18]:
samples_genetic_correlation=get_correlations(reformat(samples4G));

In [19]:
writedlm("out.G.txt",reformat(reformat(samples4G)))

In [20]:
out=readdlm("out.G.txt")

4×1000 Array{Float64,2}:
  1.71949   4.85577  138.748     5.09047  …  32.9766     59.4607    123.882
 -2.85418  -3.63175   -3.25707  -1.5976      -0.799502   -1.30149     0.0  
 -2.85418  -3.63175   -3.25707  -1.5976      -0.799502   -1.30149     0.0  
 10.493     7.01324    3.18312   3.47961      0.0591678   0.056182    0.0  

In [21]:
reformat(out)

1000-element Array{Array{Float64,2},1}:
 [1.71949 -2.85418; -2.85418 10.493]     
 [4.85577 -3.63175; -3.63175 7.01324]    
 [138.748 -3.25707; -3.25707 3.18312]    
 [5.09047 -1.5976; -1.5976 3.47961]      
 [0.962175 2.58458; 2.58458 7.05113]     
 [10.5938 -6.70172; -6.70172 4.67384]    
 [1.83497 -2.22491; -2.22491 7.81143]    
 [2.06643 3.64146; 3.64146 7.46976]      
 [62.6331 9.11548; 9.11548 1.42734]      
 [3.22563 0.971812; 0.971812 0.519603]   
 [4.39069 -2.38524; -2.38524 1.52639]    
 [0.792482 0.269443; 0.269443 0.281503]  
 [0.679894 0.0; 0.0 0.0]                 
 ⋮                                       
 [10.8962 -1.05419; -1.05419 0.208341]   
 [37.3247 0.0; 0.0 0.0]                  
 [45.7475 5.25519; 5.25519 0.740888]     
 [72.8226 11.3793; 11.3793 3.05617]      
 [53.0627 2.88175; 2.88175 1.07715]      
 [58.0952 9.48506; 9.48506 2.54233]      
 [128.651 26.5354; 26.5354 5.54279]      
 [57.5224 5.80412; 5.80412 1.13232]      
 [40.6044 -1.15771; -1.15771 1.39814

In [22]:
out10=reformat(samples4h2)

2×1000 Array{Float64,2}:
 0.00347939  0.00846272  0.292829  …  0.346202   0.150918    0.647921
 0.474342    0.240628    0.30085      0.0113355  0.00290066  0.0     

In [23]:
reformat(out,2)

1000-element Array{Array{Float64,2},1}:
 [1.71949 -2.85418; -2.85418 10.493]     
 [4.85577 -3.63175; -3.63175 7.01324]    
 [138.748 -3.25707; -3.25707 3.18312]    
 [5.09047 -1.5976; -1.5976 3.47961]      
 [0.962175 2.58458; 2.58458 7.05113]     
 [10.5938 -6.70172; -6.70172 4.67384]    
 [1.83497 -2.22491; -2.22491 7.81143]    
 [2.06643 3.64146; 3.64146 7.46976]      
 [62.6331 9.11548; 9.11548 1.42734]      
 [3.22563 0.971812; 0.971812 0.519603]   
 [4.39069 -2.38524; -2.38524 1.52639]    
 [0.792482 0.269443; 0.269443 0.281503]  
 [0.679894 0.0; 0.0 0.0]                 
 ⋮                                       
 [10.8962 -1.05419; -1.05419 0.208341]   
 [37.3247 0.0; 0.0 0.0]                  
 [45.7475 5.25519; 5.25519 0.740888]     
 [72.8226 11.3793; 11.3793 3.05617]      
 [53.0627 2.88175; 2.88175 1.07715]      
 [58.0952 9.48506; 9.48506 2.54233]      
 [128.651 26.5354; 26.5354 5.54279]      
 [57.5224 5.80412; 5.80412 1.13232]      
 [40.6044 -1.15771; -1.15771 1.39814

In [24]:
#genetic correlation between trait 1 and trait 2
report(reformat(samples4G),index=[1,2]);

Summary Stats:
Mean:           15.681404
Minimum:        -12.265450
1st Quartile:   0.000000
Median:         1.107843
3rd Quartile:   22.447004
Maximum:        180.474276
Length:         1000
Type:           Float64
nothing


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. Multiple Traits Analyses with Marker Effects and Polygenic Effects</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [25]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";
model2          = build_model(model_equations,R);

set_covariate(model2,"age");

get pedigree information from a file

In [26]:
ped=get_pedigree(pedfile);

calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00

Finished!


In [27]:
GA = G*0.1
set_random(model2,"Animal",ped,GA)

In [28]:
GM = G*0.9
add_genotypes(model2,genofile,GM,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [29]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out2=runMCMC(model2,data,Pi=Pi,chain_length=5000,methods="BayesC");

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[15.7808 1.57808; 1.57808 1.57808].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.25,[1.0, 0.0]=>0.25,[1.0, 1.0]=>0.25,[0.0, 0.0]=>0.25)
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [30]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"

In [31]:
out2["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
 1.97665   0.096068
 0.096068  0.197881